In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")

In [3]:
ME="me"

In [4]:
bought=pd.read_csv("auction.csv")
dictOfMyPlayers={}
dictOfPlayersBelongingToOtherBidders={}
for idx,player in bought.iterrows():
    if player["Owner"]==ME:
        dictOfMyPlayers[player["Name"]]=player["Price"]
    else:
        dictOfPlayersBelongingToOtherBidders[player["Name"]]=player["Price"]
print(bought.groupby("Owner").sum())
print(bought.groupby("Owner").count())

            Price
Owner            
bryan         419
fedeg         102
fedemiglia    380
gatti         433
marco         393
me            404
na              7
            Name  Price
Owner                  
bryan         14     14
fedeg          2      2
fedemiglia    13     13
gatti         17     17
marco         21     21
me            19     19
na             7      7


In [5]:
ampl.read("online-auction.mod")
ampl.readData("fantasy-auction-mantra-online.dat")

	online-auction.mod
	line 39 offset 1150
	Caution: 0-dimensional slice


In [6]:
def suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders):
    plas=ampl.getSet("Players").getValues()
    valuesForOtherPlayersAmpl={}
    valuesForMyPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForMyPlayersAmpl[name]=0
        valuesForOtherPlayersAmpl[name]=0
    budget=500
    for player,cost in dictOfPlayersBelongingToOtherBidders.items():
        valuesForOtherPlayersAmpl[player]=1
    for player,cost in dictOfMyPlayers.items():
        budget-=cost
        valuesForMyPlayersAmpl[player]=1
        
    if valuesForOtherPlayersAmpl:
        ampl.getParameter("alreadyBoughtByOther").setValues(valuesForOtherPlayersAmpl)
    if valuesForMyPlayersAmpl:
        ampl.getParameter("alreadyBoughtByMe").setValues(valuesForMyPlayersAmpl)
        
        
    
    ampl.getParameter("budget").set(budget)
    ampl.solve()


In [7]:
suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders)

CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 89.54005882
29168 MIP simplex iterations
95 branch-and-bound nodes
absmipgap = 0.00712919, relmipgap = 7.96201e-05


In [8]:
buy=ampl.getVariable("putPlayerInSlot")
cost=ampl.getParameter("cost")
pfs=ampl.getParameter("performance")
moduleChosen=ampl.getVariable("bidderChoosesModule")

roles=ampl.getSet("CompatibilityPlayerRole").getValues().toDict()

solution=buy.getValues().toDict()
costs=cost.getValues().toDict()
pfs=pfs.getValues().toDict()
modules=moduleChosen.getValues().toDict()
moduleUsed="AAA"
for moduleName,taken in modules.items():
    if taken>0.5:
        print("Module: {}\n\n".format(moduleName))
        moduleUsed=moduleName

myPls=[]
for key,val in solution.items():
    if key[0]==moduleName:
        if val>0.5:
            myPls.append(key[1])
rolesA={}

for key,val in roles.items():
    if key[0] in myPls:
        if key[0] not in rolesA.keys():
            rolesA[key[0]]=[]
        rolesA[key[0]].append(key[1])
print(rolesA)

Module: m442


{'ZAPATAD': ['Pc'], 'QUAGLIARELLA': ['Pc'], 'GOMEZA': ['T', 'A'], 'IZZO': ['Dd', 'Dc'], 'ZIELINSKI': ['C', 'T'], 'POLITANO': ['W', 'A'], 'NKOULOU': ['Dc'], 'KESSIE': ['C'], 'KOULIBALY': ['Dc'], 'CRISCITO': ['Ds', 'Dc', 'E'], 'LAZOVIC': ['E', 'W'], 'VERETOUT': ['C'], 'KURTIC': ['C', 'T'], 'SIRIGU': ['Por'], 'MURRU': ['Ds', 'E'], 'FLORENZI': ['Dd', 'E'], 'BONUCCI': ['Dc'], 'BONIFAZI': ['Dc'], 'PISACANE': ['Dd', 'Ds', 'Dc'], 'CASTILLEJO': ['W', 'A'], 'LAMANTIA': ['Pc'], 'PAZZINI': ['Pc'], 'OLSEN': ['Por'], 'SANDRO': ['M', 'C'], 'MOROSINI': ['T', 'A'], 'CASTROVILLI': ['C', 'T'], 'AMRABAT': ['E', 'M'], 'RRAHMANI': ['Dd', 'Ds', 'Dc']}


In [9]:
for player in myPls:
    message="{}                 {}   {}  ".format(player,costs[player],rolesA[player])
    print(message)

AMRABAT                 1.0   ['E', 'M']  
BONIFAZI                 17.358906357286802   ['Dc']  
BONUCCI                 22.9064237043878   ['Dc']  
CASTILLEJO                 10.3309568660778   ['W', 'A']  
CASTROVILLI                 1.0   ['C', 'T']  
CRISCITO                 33.1118860902676   ['Ds', 'Dc', 'E']  
FLORENZI                 23.0968118736965   ['Dd', 'E']  
GOMEZA                 50.8649843671469   ['T', 'A']  
IZZO                 41.974783627056   ['Dd', 'Dc']  
KESSIE                 36.9130843671568   ['C']  
KOULIBALY                 35.52456375306   ['Dc']  
KURTIC                 29.501075982687002   ['C', 'T']  
LAMANTIA                 2.55969418097252   ['Pc']  
LAZOVIC                 31.4903040964996   ['E', 'W']  
MOROSINI                 1.0   ['T', 'A']  
MURRU                 28.2077494532446   ['Ds', 'E']  
NKOULOU                 37.59585573295389   ['Dc']  
OLSEN                 2.11472259401068   ['Por']  
PAZZINI                 2.55969418097252  